In [ ]:
#Imports
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')




In [3]:
df = pd.read_csv("../data/kickstarter_projects.csv")
df.head()

,ID,Name,Category,Subcategory,Country,Launched,Deadline,Goal,Pledged,Backers,State
0,1860890148,Grace Jones Does Not Give A F$#% T-Shirt (limi...,Fashion,Fashion,United States,2009-04-21 21:02:48,2009-05-31,1000,625,30,Failed
1,709707365,CRYSTAL ANTLERS UNTITLED MOVIE,Film & Video,Shorts,United States,2009-04-23 00:07:53,2009-07-20,80000,22,3,Failed
2,1703704063,drawing for dollars,Art,Illustration,United States,2009-04-24 21:52:03,2009-05-03,20,35,3,Successful
3,727286,Offline Wikipedia iPhone app,Technology,Software,United States,2009-04-25 17:36:21,2009-07-14,99,145,25,Successful
4,1622952265,Pantshirts,Fashion,Fashion,United States,2009-04-27 14:10:39,2009-05-26,1900,387,10,Failed


In [4]:
df.columns.value_counts

<bound method IndexOpsMixin.value_counts of Index(['ID', 'Name', 'Category', 'Subcategory', 'Country', 'Launched',
       'Deadline', 'Goal', 'Pledged', 'Backers', 'State'],
      dtype='object')>

In [5]:
df.nunique

<bound method DataFrame.nunique of                 ID                                               Name   
0       1860890148  Grace Jones Does Not Give A F$#% T-Shirt (limi...  \
1        709707365                     CRYSTAL ANTLERS UNTITLED MOVIE   
2       1703704063                                drawing for dollars   
3           727286                       Offline Wikipedia iPhone app   
4       1622952265                                         Pantshirts   
...            ...                                                ...   
374848  1486845240                  Americas Got Talent - Serious MAK   
374849   974738310  EVO Planner: The World's First Personalized Fl...   
374850  2106246194    Help save La Gattara, Arizona's first Cat Cafe!   
374851  1830173355                                Digital Dagger Coin   
374852  1339173863                              Spirits of the Forest   

            Category     Subcategory        Country             Launched   
0           